# Assignment 9 Neural Nets


## Perceptron
To start we are going to examine a classical Perceptron.  

In [2]:
## Lets start with simple logic functions of two arguments 
# A with B
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier


logic_data = np.array([
    [1, 1, 1, 1, 0],
    [1, 0, 0, 1, 1],
    [0, 1, 0, 1, 1],
    [0, 0, 0, 0, 0],
])

# slice it into a dictionary
d = {}
d['A'] = logic_data[:,0]  #This is the first column
d['B'] = logic_data[:,1]  
d['AND'] = logic_data[:,2]  
d['OR'] = logic_data[:,3]  
d['XOR'] = logic_data[:,4]  
logic_df = pd.DataFrame(d)

print(logic_df)


   A  B  AND  OR  XOR
0  1  1    1   1    0
1  1  0    0   1    1
2  0  1    0   1    1
3  0  0    0   0    0


## Part 1
Create and train a MLPClassifier on the logic data frame.  Since this is a perceptron, there will be no hidden layer 
* Use hidden_layer_sizes = () This is an empty tuple of sizes.
* Use 'logistic' as the activation.  
* Set the maximum number of iterations to 1000 and increase by 1000 until you get convergence.
* Use an initial_learning_rate = 0.01  (You can try changing this.)
* Use ['A','B'] for X
* Use 'AND' for y

In [3]:

# your code here
x = logic_df[["A","B"]]
y = logic_df['AND']

mlp = MLPClassifier(
    solver='adam', 
    activation='logistic', 
    alpha=1e-5, 
    hidden_layer_sizes=(),
    random_state=1,max_iter=1000,
    initail_learning_rate = 0.01)
mlp.fit(x,y)

TypeError: __init__() got an unexpected keyword argument 'initail_learning_rate'

 using initial_learning_rate = 0.01  gave an error so imma try it without it.

In [4]:
x = logic_df[["A","B"]]
y = logic_df['AND']

mlp = MLPClassifier(
    solver='adam', 
    activation='logistic', 
    alpha=1e-5, 
    hidden_layer_sizes=(),
    random_state=1,max_iter=1000)
mlp.fit(x,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

## Part 2 
Compute a confusion matrix on the logic data frame and see how you did

In [5]:
y_predicted = mlp.predict(x)
matrix = confusion_matrix(y, y_predicted)
print(matrix)

[[3 0]
 [0 1]]


## Part 3
Train the model and then compute a confusion matrix using 'OR' for y

In [6]:
mlp.fit(x,y)
y = logic_df["OR"]
y_predicted = mlp.predict(x)
matrix = confusion_matrix(y, y_predicted)
print(matrix)

[[1 0]
 [2 1]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Part 4
Train the model and then compute a confusion matrix using 'XOR' for y.
Print the predicted values of y.


In [10]:
mlp.fit(x,y)
y = logic_df["XOR"]
y_predicted = mlp.predict(x)
matrix = confusion_matrix(y, y_predicted)
print(matrix)
print("*************************************************")
print(y_predicted)

[[2 0]
 [2 0]]
*************************************************
[0 0 0 0]


### Fit this multiple times and remark on the results here:
ran the fit 100 times and the result was the same.

In [12]:
for n in range(100):
    mlp.fit(x,y)
y = logic_df["XOR"]
y_predicted = mlp.predict(x)
matrix = confusion_matrix(y, y_predicted)
print(matrix)
print("*************************************************")
print(y_predicted)

[[2 0]
 [2 0]]
*************************************************
[0 0 0 0]


## Part 5
Create a new multi-layer-perceptron with a single hidden layer with four nodes.
* hidden_layer_sizes = (4)


In [14]:
mlp2 = MLPClassifier(
    solver='adam', 
    activation='logistic', 
    alpha=1e-5, 
    hidden_layer_sizes=(4),
    random_state=1,max_iter=1000)


## Part 6
Train the new model and then compute a confusion matrix using 'XOR' for y.
Print the predicted values of y.


In [15]:
mlp2.fit(x,y)
y = logic_df["XOR"]
y_predicted = mlp.predict(x)
matrix = confusion_matrix(y, y_predicted)
print(matrix)
print("*************************************************")
print(y_predicted)

[[2 0]
 [2 0]]
*************************************************
[0 0 0 0]


## About the Balance dataset
The data set you will be using in the second part is the same set we used last time the UCI Machine Learning DataSet repository.  
http://mlr.cs.umass.edu/ml/datasets/Balance+Scale

It is a Balance scale weight & distance database.  You have a left weight and distance and a right weight and distance.  There are three categories:
* B is balanced
* L is scale tips left
* R is scale tips right

For the purposses of this assignment, we are going to define balance, we are going to define the categories slightly differently.  We are not going to look for perfect balance, instead we are going to look for a "close" balance.  Still compute  left weight*distance and right weight * distance.  If absolute value of the difference is less than 0.5, we will tag it as balanced.  Otherwise if the left product is greater, then we have a tip left, equal is balanced, right greater is tip right.

This data set exhaustively gives examples for all 625 posible combinations of weights of 1, 2, 3, 4, 5 with distances of 1, 2, 3, 4, 5.

If generating new instances, we should restrict the weight and distance to be positive.

## Given: Get the DataSet and Cross Validate
We use a decision tree to give us a base line. We cross validate because the decision tree will overfit the data.

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# Get the data setimport pandas as pd
balance_df = pd.read_csv("balance-scale.csv")

# Add new features
balance_df["Left-Product"]=balance_df["Left-Weight"]*balance_df["Left-Distance"]
balance_df["Right-Product"]=balance_df["Right-Weight"]*balance_df["Right-Distance"]

# Use a decision tree classifier and cross validate
from sklearn.tree import DecisionTreeClassifier
X = balance_df[["Left-Weight", 'Left-Distance', "Right-Weight", "Right-Distance"]]
y = balance_df['Class']

from sklearn.model_selection import KFold
validation_accuracy = []
validation_f1 =[]
fold_and_validate = KFold(n_splits=5, shuffle=True, random_state=145)
for train_set_indices, validation_set_indices in fold_and_validate.split(X):
    cv_train_set = X.iloc[train_set_indices]
    cv_train_target = y.iloc[train_set_indices]
    
    cv_decision_tree = DecisionTreeClassifier()
    cv_decision_tree.fit(cv_train_set, cv_train_target)
    
    cv_xvalidation = X.iloc[validation_set_indices]
    cv_y_true = y.iloc[validation_set_indices]
    cv_y_predicted = cv_decision_tree.predict(cv_xvalidation)
    
    cv_accuracy_score = accuracy_score(cv_y_true, cv_y_predicted)
    cv_f1_score = f1_score(cv_y_true, cv_y_predicted,  average="weighted")
    validation_accuracy.append(cv_accuracy_score)
    validation_f1.append(cv_f1_score)
    
print("Cross validation accuracies are: ", validation_accuracy)
print("Cross validation f1 scores  are: ", validation_f1)


# 

Cross validation accuracies are:  [0.76, 0.736, 0.824, 0.776, 0.824]
Cross validation f1 scores  are:  [0.77651376146789, 0.7640212201591512, 0.8171428571428571, 0.7894983077528532, 0.8345003830243605]


## Given: - Create test sets 
We will create two test sets
```integer_10_test_set```  This set will have integer weights and distances randomly chosen from 1 to 10.  

```float_5_test_set```  This set will have floating point weights and distances randomly chosen from 1 to 5.

The balance function we will use is not exact balance, but will look for a close balance.  The reason for this is that if we use exact balance we will not get anything in the balanced category for the floating point set.  

We print the counts to guarantee that we have enough in the balance category.

In [2]:
import numpy as np
import pandas as pd


# A function that computes balance
def close_balance(left, right):
    if (abs(left-right) < 0.5): return "B"
    if left<right: return "R"
    return "L"

# Generate the same sequence each time
np.random.seed(20)

# create my data values for integer_10_test_set
# create 10000 instances
integer_10_leftW = np.random.randint(1, 10, 10000)     
integer_10_leftD = np.random.randint(1, 10, 10000)    
integer_10_rightW = np.random.randint(1, 10, 10000)    
integer_10_rightD = np.random.randint(1, 10, 10000)  
integer_10_leftP = integer_10_leftW * integer_10_leftD
integer_10_rightP = integer_10_rightW * integer_10_rightD

integer_10_target = [close_balance(left,right) for (left,right) in zip(integer_10_leftP, integer_10_rightP)]

# create a dictionary with each feature
d = {}
d["Class"] = integer_10_target
d["Left-Weight"] = integer_10_leftW
d["Left-Distance"] = integer_10_leftD
d["Right-Weight"] = integer_10_rightW
d["Right-Distance"] = integer_10_rightD
d["Left-Product"] = integer_10_leftP
d["Right-Product"] = integer_10_rightP

# Create the data frame from the dictionary
integer_10_test_set = pd.DataFrame(data=d)

print(integer_10_test_set["Class"].value_counts())


# create my data values for float_5_test_set
# create 10000 instances
float_5_leftW = np.random.uniform(1, 5, 10000)     
float_5_leftD = np.random.uniform(1, 5, 10000)    
float_5_rightW = np.random.uniform(1, 5, 10000)    
float_5_rightD = np.random.uniform(1, 5, 10000)  
float_5_leftP = float_5_leftW * float_5_leftD
float_5_rightP = float_5_rightW * float_5_rightD

float_5_target = [close_balance(left,right) for (left,right) in zip(float_5_leftP, float_5_rightP)]

# create a dictionary with each feature
d = {}
d["Class"] = float_5_target
d["Left-Weight"] = float_5_leftW
d["Left-Distance"] = float_5_leftD
d["Right-Weight"] = float_5_rightW
d["Right-Distance"] = float_5_rightD
d["Left-Product"] = float_5_leftP
d["Right-Product"] = float_5_rightP


# Create the data frame from the dictionary
float_5_test_set = pd.DataFrame(data=d)
print(float_5_test_set["Class"].value_counts())

      
      
      


R    4843
L    4831
B     326
Name: Class, dtype: int64
L    4704
R    4686
B     610
Name: Class, dtype: int64


## Given - Train on Balance Data Set and Test our new sets


In [34]:
from sklearn.tree import DecisionTreeClassifier

# Train the classifier
X = balance_df[["Left-Weight", 'Left-Distance', "Right-Weight", "Right-Distance"]]
y = balance_df['Class']
tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(X,y)

# Test it on our two test sets
print("Evaluating Decision Tree on the integers from 1 to 10 test set")
X_test_int_10 = integer_10_test_set[["Left-Weight", 'Left-Distance', "Right-Weight", "Right-Distance"]]
y_true = integer_10_test_set["Class"]

y_predicted = tree_classifier.predict(X_test_int_10)
matrix = confusion_matrix(y_true, y_predicted)
print(matrix)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print ("Accuracy is ", accuracy_score(y_true, y_predicted))

print ("F1 is ", f1_score(y_true, y_predicted, average="weighted"))

print()
print("Evaluating Decision Tree on the floats from 1 to 5 test set")
X_test_float_5 = float_5_test_set[["Left-Weight", 'Left-Distance', "Right-Weight", "Right-Distance"]]
y_true = float_5_test_set["Class"]

y_predicted = tree_classifier.predict(X_test_float_5)
matrix = confusion_matrix(y_true, y_predicted)
print(matrix)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print ("Accuracy is ", accuracy_score(y_true, y_predicted))

print ("F1 is ", f1_score(y_true, y_predicted, average="weighted"))



/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Evaluating Decision Tree on the integers from 1 to 10 test set
[[   0  200  126]
 [   0 4702  129]
 [   0  128 4715]]
Accuracy is  0.9417
F1 is  0.9261089642329201

Evaluating Decision Tree on the floats from 1 to 5 test set
[[   0  299  311]
 [   0 4657   47]
 [   0   52 4634]]
Accuracy is  0.9291
F1 is  0.8998710706154994


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Part 7 - Train and Test a Neural Net
Use the four features Left-Weight, Left-Distance, Right-Weight, and Right-Distance.  Use Class as the target.

* Use a single hidden layer of size 20
* Use 'logistic' as the activation.  
* Set the maximum number of iterations to 1000 and increase by 1000 until you get convergence.
* Use an initial_learning_rate = 0.01  (You can try changing this.)


Get the performance measures for the two sets.


In [5]:
## Your code



### Your comparison with previous results here:

## Part 8 - Vary size of hidden layer.
Use sizes for the hidden layer of 10, 20, 30, 100, 200, 300.
Report the confusion matrix and accuracy for each.


In [4]:
## Your code

### As you change the size of the hidden layer how does the performance change:

## Part 9 - Vary the number of hidden layers.  
Set the sizes for the hidden layers to 
* ()
* (50)
* (50, 50)
* (50, 50, 50)
* (50, 50, 50, 50)

Report the confusion matrix and accuracy for each.



In [4]:
## Your code

### As you change the number of hidden layers how does the performance change:

## Part 10 - Vary solver and activation

Set the hidden layers to  (500, 500) and use each of the following combinations
* solver='adam', activation='logistic'
* solver='adam', activation='relu'
* solver='sgd', activation='logistic'
* solver='sgd', activation='relu'

As well as performance, look at how quickly the network trains.
  

In [5]:
# Your code


### Which combination worked the best:

## Bonus
Explore the capabilities of neural nets...
1. Try different sizes for the hidden layers. We can try decreasing sizes for the hidden layers, increasing sizes, or alternating sizes.  For this data set, which one worked the best?
1. Try using other parameter combinations
* solver can be 'sgd', 'adam', 'lbfgs'
* activation can be 'logistic', 'relu', 'tanh'
* batch_size can be an int (Does minibatch with adam or sgd)
* learning_rate can be ‘constant’, ‘invscaling’, ‘adaptive’